In [17]:
import numpy as np
import tensorflow as tf

from tensorflow import keras

2023-04-26 06:32:41.813546: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
def encode_haar(inputs):
  outputs = []
  idx = 0

  while(inputs.shape[1] > 1):
    odds = inputs[:,::2]
    evens = inputs[:,1::2]
    delta = evens - odds
    print(delta)
    idx_end = idx + tf.shape(delta)[1]
    print(tf.shape(delta)[1])
    outputs.append(delta)
    idx = idx_end
    inputs = (odds + evens) / 2

  outputs.append(inputs)
  outputs = tf.concat(outputs, axis=1)
  return outputs

In [14]:
def decode_haar(inputs):
  print(inputs)
  previous_inputs = inputs[:,-1:]
  inputs_len = inputs.shape[1]
  idx = 1
  while (idx < inputs_len):
    outputs = []
    size = idx * 2
    inputs_range = inputs[:, inputs_len-size:inputs_len-idx]

    for i in range(inputs_range.shape[1]):
      print(inputs_range)
      half_input_to_add = inputs_range[:,i:i+1] / 2
      left = previous_inputs[:,i:i+1] - half_input_to_add
      right = previous_inputs[:,i:i+1] + half_input_to_add
      outputs.append(left)
      outputs.append(right)

    outputs = tf.concat(outputs, axis=1)
    previous_inputs = outputs
    idx = size
  return outputs

In [18]:
input_layer = keras.Input(shape=(4,))

# Encode
encode_model = keras.Model(input_layer, encode_haar(input_layer))

# Decode
decode_model = keras.Model(input_layer, decode_haar(input_layer))

KerasTensor(type_spec=TensorSpec(shape=(None, 2), dtype=tf.float32, name=None), name='tf.math.subtract/Sub:0', description="created by layer 'tf.math.subtract'")
KerasTensor(type_spec=TensorSpec(shape=(), dtype=tf.int32, name=None), inferred_value=[2], name='tf.__operators__.getitem_3/strided_slice:0', description="created by layer 'tf.__operators__.getitem_3'")
KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='tf.math.subtract_1/Sub:0', description="created by layer 'tf.math.subtract_1'")
KerasTensor(type_spec=TensorSpec(shape=(), dtype=tf.int32, name=None), inferred_value=[1], name='tf.__operators__.getitem_7/strided_slice:0', description="created by layer 'tf.__operators__.getitem_7'")
KerasTensor(type_spec=TensorSpec(shape=(None, 4), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='tf.__operators__.getitem_9/strided_sl

In [19]:
inputs = np.array([[2, 1, 0, -2, 2.5, 0, 1.75, 7.125]])
print(encode_model.predict(inputs))

ValueError: in user code:

    File "/usr/local/lib/python3.8/dist-packages/keras/src/engine/training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.8/dist-packages/keras/src/engine/training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.8/dist-packages/keras/src/engine/training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/lib/python3.8/dist-packages/keras/src/engine/training.py", line 2283, in predict_step
        return self(x, training=False)
    File "/usr/local/lib/python3.8/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.8/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Inp

In [8]:
inputs = np.array([[9, 7, 3, 5]])
print(decode_model.predict(inputs))

1/1 [==============================] - 0s 112ms/step
[[-1.  8.  3. 10.]]
